## Échanger quantiquement une clé

### Alice - Préparation/envoi

1. |1⟩ + H
1. |0⟩ 
1. |1⟩ + H
1. |0⟩ + H
1. |1⟩
1. |1⟩ + H
1. |0⟩
1. |1⟩ + H

### Bob - Reçoit/traite/demande

#### Réception/traitement

<table>
<tr><th></th><th>Reçu</th><th>Aléatoire</th><th>Mesure</th></tr>
<tr><td>1</td><td>|1⟩ + H</td><td></td><td>0/1</td></tr>
<tr><td>2</td><td>|0⟩ </td><td>H</td><td>0/1</td></tr>
<tr><td>3</td><td>|1⟩ + H</td><td></td><td>0/1</td></tr>
<tr><td>4</td><td>|0⟩ + H</td><td>H</td><td>0</td></tr>
<tr><td>5</td><td>|1⟩</td><td></td><td>1</td></tr>
<tr><td>6</td><td>|1⟩ + H</td><td></td><td>0/1</td></tr>
<tr><td>7</td><td>|0⟩</td><td>H </td><td>0/1</td></tr>
<tr><td>8</td><td>|1⟩ + H</td><td>H</td><td>1</td></tr>
</table>

#### Envoi "J'ai fait H sur... Que garder ?"

J'ai fait H sur **2, 4, 7, 8** Que garder ? 

### Alice - Répond

<table>
<tr><th></th><th>A</th><th>B</th><th></th></tr>
<tr><td>1</td><td>H</td><td> </td><td></td></tr>
<tr><td>2</td><td> </td><td>H</td><td></td></tr>
<tr><td>3</td><td>H</td><td> </td><td></td></tr>
<tr><td>4</td><td>H</td><td>H</td><td>ok</td></tr>
<tr><td>5</td><td> </td><td> </td><td>ok</td></tr>
<tr><td>6</td><td>H</td><td> </td><td></td></tr>
<tr><td>7</td><td> </td><td>H</td><td></td></tr>
<tr><td>8</td><td>H</td><td>H</td><td>ok</td></tr>
</table>
Message à Bob : 4, 5, 8

Clé est : 0,  1, 1

### Bob déduit la clé

Clé est : 0, 1, 1